In [38]:
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity

In [95]:
# Authentication credentials
client_id = '67dd441a7f8249ecb919eb72ea11f460' 
client_secret = '3a521e0f708f44fe8625f92cfeaef485'
market = 'US'

# Create Spotify object
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

# Query artist
seed_artists = [
    '4exLIFE8sISLr28sqG1qNX', # travis barker
    '4q3ewBCX7sLwd24euuV69X', # bad bunny
    '5he5w2lnU9x7JFhnwcekXX' # Skrillex
    ] 

# Initialize dataframe to store data
artists_df = pd.DataFrame(columns=['id', 'name', 'genre']) 

# Iterate through seed artists
for artist_id in seed_artists:

  # Get artist name
  artist = sp.artist(artist_id)
  name = artist['name']
  id = artist['id']
  genres = artist['genres']
  popularity = artist['popularity']
  
  artists_df = artists_df._append({
                            'id': id,
                            'name': name,
                            'genre': genres,
                            'popularity': popularity
                            }, ignore_index=True)

print(artists_df.head())





                       id           name  \
0  4exLIFE8sISLr28sqG1qNX  Travis Barker   
1  4q3ewBCX7sLwd24euuV69X      Bad Bunny   
2  5he5w2lnU9x7JFhnwcekXX       Skrillex   

                                            genre  popularity  
0                                      [rap rock]        65.0  
1         [reggaeton, trap latino, urbano latino]        95.0  
2  [brostep, complextro, edm, electro, pop dance]        74.0  


In [96]:
artists_df

,id,name,genre,popularity
0,4exLIFE8sISLr28sqG1qNX,Travis Barker,[rap rock],65.0
1,4q3ewBCX7sLwd24euuV69X,Bad Bunny,"[reggaeton, trap latino, urbano latino]",95.0
2,5he5w2lnU9x7JFhnwcekXX,Skrillex,"[brostep, complextro, edm, electro, pop dance]",74.0


In [92]:
tracks_data = []

for artist_id in seed_artists:
    results = sp.artist_top_tracks(artist_id, country='US')
    
    for track in results['tracks']:
        track_id = track['id']
        track_name = track['name']
        
        track_data = {
            'artist_id': artist_id,
            'track_id': track_id,
            'track_name': track_name
        }
        
        tracks_data.append(track_data)

# print(tracks_data)

tracks_df = pd.DataFrame(tracks_data, columns=['artist_id', 'track_id', 'track_name'])

[{'artist_id': '4exLIFE8sISLr28sqG1qNX', 'track_id': '2gTdDMpNxIRFSiu7HutMCg', 'track_name': "I Think I'm OKAY (with YUNGBLUD & Travis Barker)"}, {'artist_id': '4exLIFE8sISLr28sqG1qNX', 'track_id': '5JKnYZOhSP7rOK0jPYcCjV', 'track_name': 't r a n s p a r e n t s o u l feat. Travis Barker'}, {'artist_id': '4exLIFE8sISLr28sqG1qNX', 'track_id': '59AziPyNm2kHqyd9lH9cSr', 'track_name': 'Sick and Tired (feat. Machine Gun Kelly & Travis Barker)'}, {'artist_id': '4exLIFE8sISLr28sqG1qNX', 'track_id': '556awMv4WQP0h1VXMU8rSU', 'track_name': 'Eyes Wide Shut'}, {'artist_id': '4exLIFE8sISLr28sqG1qNX', 'track_id': '4XBDGFxecM5Y3KZFF0iufe', 'track_name': 'Aliens Are Ghosts'}, {'artist_id': '4exLIFE8sISLr28sqG1qNX', 'track_id': '3emo5sUq2OwBctNgX0NMEV', 'track_name': 'estella//'}, {'artist_id': '4exLIFE8sISLr28sqG1qNX', 'track_id': '5YEHrYUjG9e0VPQLCp6vGo', 'track_name': 'Misery Business'}, {'artist_id': '4exLIFE8sISLr28sqG1qNX', 'track_id': '0SbcVQyMO5M68AzhX8gYaS', 'track_name': 'SIDEWAYS (with Trav

In [93]:
tracks_df

,artist_id,track_id,track_name
0,4exLIFE8sISLr28sqG1qNX,2gTdDMpNxIRFSiu7HutMCg,I Think I'm OKAY (with YUNGBLUD & Travis Barker)
1,4exLIFE8sISLr28sqG1qNX,5JKnYZOhSP7rOK0jPYcCjV,t r a n s p a r e n t s o u l feat. Travis Barker
2,4exLIFE8sISLr28sqG1qNX,59AziPyNm2kHqyd9lH9cSr,Sick and Tired (feat. Machine Gun Kelly & Trav...
3,4exLIFE8sISLr28sqG1qNX,556awMv4WQP0h1VXMU8rSU,Eyes Wide Shut
4,4exLIFE8sISLr28sqG1qNX,4XBDGFxecM5Y3KZFF0iufe,Aliens Are Ghosts
5,4exLIFE8sISLr28sqG1qNX,3emo5sUq2OwBctNgX0NMEV,estella//
6,4exLIFE8sISLr28sqG1qNX,5YEHrYUjG9e0VPQLCp6vGo,Misery Business
7,4exLIFE8sISLr28sqG1qNX,0SbcVQyMO5M68AzhX8gYaS,SIDEWAYS (with Travis Barker)
8,4exLIFE8sISLr28sqG1qNX,3gr1pMLpDJGqErHhhl7dEO,thought it was
9,4exLIFE8sISLr28sqG1qNX,0vJ3WZ0Q6AdntTxQ384JQN,Let's Go


In [118]:

# Initialize empty dataframe for audio features
features_df = pd.DataFrame() 

# Iterate through track IDs
for index, row in tracks_df.iterrows():

  track_id = row['track_id']
  
  # Get audio features
  features = sp.audio_features(track_id)[0]
  
  # Extract relevant features
  danceability = features['danceability']
  energy = features['energy']
  key = features['key']
  loudness = features['loudness']
  tempo = features['tempo']
  
  # Add features to dataframe
  track_features = {
    'track_id': track_id,
    'danceability': danceability,
    'energy': energy, 
    'key': key,
    'loudness': loudness,
    'tempo': tempo
  }
  
  features_df = features_df._append(track_features, ignore_index=True)

print(features_df.head())

                 track_id  danceability  energy  key  loudness    tempo
0  2gTdDMpNxIRFSiu7HutMCg         0.628   0.744    7    -4.718  119.921
1  5JKnYZOhSP7rOK0jPYcCjV         0.374   0.830   11    -5.203   90.190
2  59AziPyNm2kHqyd9lH9cSr         0.747   0.486   11    -8.810  145.089
3  556awMv4WQP0h1VXMU8rSU         0.573   0.848    4    -3.413   89.978
4  4XBDGFxecM5Y3KZFF0iufe         0.760   0.624   11    -7.803  139.032


In [119]:
features_df

,track_id,danceability,energy,key,loudness,tempo
0,2gTdDMpNxIRFSiu7HutMCg,0.628,0.744,7,-4.718,119.921
1,5JKnYZOhSP7rOK0jPYcCjV,0.374,0.830,11,-5.203,90.190
2,59AziPyNm2kHqyd9lH9cSr,0.747,0.486,11,-8.810,145.089
3,556awMv4WQP0h1VXMU8rSU,0.573,0.848,4,-3.413,89.978
4,4XBDGFxecM5Y3KZFF0iufe,0.760,0.624,11,-7.803,139.032
5,3emo5sUq2OwBctNgX0NMEV,0.454,0.837,8,-4.395,164.966
6,5YEHrYUjG9e0VPQLCp6vGo,0.461,0.867,11,-3.586,170.015
7,0SbcVQyMO5M68AzhX8gYaS,0.606,0.811,1,-4.785,90.076
8,3gr1pMLpDJGqErHhhl7dEO,0.695,0.504,4,-6.090,90.018
9,0vJ3WZ0Q6AdntTxQ384JQN,0.710,0.905,9,-5.804,150.002


In [128]:
artists_df.keys()

Index(['id', 'name', 'genre', 'popularity'], dtype='object')

In [129]:
merged_df = pd.merge(features_df, tracks_df, on='track_id')

merged_df

,track_id,danceability,energy,key,loudness,tempo,artist_id,track_name
0,2gTdDMpNxIRFSiu7HutMCg,0.628,0.744,7,-4.718,119.921,4exLIFE8sISLr28sqG1qNX,I Think I'm OKAY (with YUNGBLUD & Travis Barker)
1,5JKnYZOhSP7rOK0jPYcCjV,0.374,0.830,11,-5.203,90.190,4exLIFE8sISLr28sqG1qNX,t r a n s p a r e n t s o u l feat. Travis Barker
2,59AziPyNm2kHqyd9lH9cSr,0.747,0.486,11,-8.810,145.089,4exLIFE8sISLr28sqG1qNX,Sick and Tired (feat. Machine Gun Kelly & Trav...
3,556awMv4WQP0h1VXMU8rSU,0.573,0.848,4,-3.413,89.978,4exLIFE8sISLr28sqG1qNX,Eyes Wide Shut
4,4XBDGFxecM5Y3KZFF0iufe,0.760,0.624,11,-7.803,139.032,4exLIFE8sISLr28sqG1qNX,Aliens Are Ghosts
5,3emo5sUq2OwBctNgX0NMEV,0.454,0.837,8,-4.395,164.966,4exLIFE8sISLr28sqG1qNX,estella//
6,5YEHrYUjG9e0VPQLCp6vGo,0.461,0.867,11,-3.586,170.015,4exLIFE8sISLr28sqG1qNX,Misery Business
7,0SbcVQyMO5M68AzhX8gYaS,0.606,0.811,1,-4.785,90.076,4exLIFE8sISLr28sqG1qNX,SIDEWAYS (with Travis Barker)
8,3gr1pMLpDJGqErHhhl7dEO,0.695,0.504,4,-6.090,90.018,4exLIFE8sISLr28sqG1qNX,thought it was
9,0vJ3WZ0Q6AdntTxQ384JQN,0.710,0.905,9,-5.804,150.002,4exLIFE8sISLr28sqG1qNX,Let's Go
